Création de la liste de tous les postes pour les Alpes. Ce cahier fournit les outils pour extraire l'information sur les postes météorologiques et nivologiques
de Météo-France.

Fonctions :
* extraire_information(département) : fournit l'information brute pour un département
* synthétiser_information(département) : fournit une liste synthétique (dataframe) des stations, à savoir :

  1. le nom, l'altitude, les coordonnées (latitude, longitude)
  2. les dates de début et de fin des mesures
  3. quel type (booléen) de variables a été mesuré :
    - pluie : pluie journalière
    - neige : cumul de neige sur 24 h
    - HN : épaisseur du manteau neigeux
    - TM : température moyenne
    - TN : température minimale
    - TX : température maximale
  
    export possible vers un fichier excel
* cartographier(liste des stations, département) : montre une carte de localisation des stations  

Les fonctions font appel à des variables globales : 
  * répertoire_travail : adresse du répertoire où les fichiers sont sauvegardés
  * export_détail, export_liste, export_carte: booléens d'export des fichiers bruts, de la liste synthétique des stations et de la carte
  * verbosity : booléen. Sert à afficher des messages de progression

# Fonctions et initialisation

In [74]:
import pandas as pd
import os
import datetime
import json
import requests
import sys
import openpyxl #export xls
import numpy as np
import gzip
from tqdm import tqdm
import plotly.express as px

################################
## Auteur : Christoophe Ancey ##
## Date : Mai 2024            ##
################################

# Création du répertoire de travail
# changer l'adresse si nécessaire
répertoire_travail = '/home/ancey/Météo-France/postes/'

# exporte le détail de chaque département
export_détail = True
export_liste  = True
export_carte  = True
verbosity     = False

# adresse url de base
base_url = 	"https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_"

# valeur à modifier si on veut afficher plus de lignes de dataframe
#pd.set_option('display.min_rows', 20)

# dictionnaire reliant numéro (ou abréviation) et nom
noms_département = dict()
noms_département['73'] = "Savoie"
noms_département['74'] = "Haute-Savoie"
noms_département['38'] = "Isère"
noms_département['05'] = "Hautes-Alpes"
noms_département['04'] = "Alpes-de-Haute-Provence"
noms_département['06'] = "Alpes-Maritimes"
noms_département['01'] = "Ain"
noms_département['25'] = "Doubs"
noms_département['26'] = "Drôme"
noms_département['nord'] = "Alpes du Nord"
noms_département['sud'] = "Alpes du Sud"
noms_département['alpes'] = "Alpes"

# début des mesures au niveau départemental
an_début = {}
an_début['01'] = '1852'
an_début['04'] = '1872'
an_début['05'] = '1877'
an_début['06'] = '1877'
an_début['25'] = '1852'
an_début['26'] = '1877'
an_début['38'] = '1872'
an_début['73'] = '1871'
an_début['74'] = '1876'
liste_départements = {'01','04','05','06','25','26','38','73','74'}
url_M1_département = {}
url_M2_département = {}
url_M3_département = {}
url_N1_département = {}
url_N2_département = {}
url_N3_département = {}
for département in liste_départements:
      url_N1_département[département] = base_url+département+"_"+an_début[département]+'-1949_autres-parametres.csv.gz'
      url_N2_département[département] = base_url+département+'_previous-1950-2022_autres-parametres.csv.gz'
      url_N3_département[département] = base_url+département+'_latest-2023-2024_autres-parametres.csv.gz'
      url_M1_département[département] = base_url+département+"_"+an_début[département]+'-1949_RR-T-Vent.csv.gz'
      url_M2_département[département] = base_url+département+'_previous-1950-2022_RR-T-Vent.csv.gz'
      url_M3_département[département] = base_url+département+'_latest-2023-2024_RR-T-Vent.csv.gz'


# teste si le répertoire existe 
if not os.path.exists(répertoire_travail):
    # crée le répertoire
    os.makedirs(répertoire_travail)
    print("J'ai créé le répertoire : ", répertoire_travail)
else:
    print("J'ai trouvé le répertoire : ",répertoire_travail)

# fonction pour télécharger les fichiers MF
def télécharger_fichier(url, nom_fichier, work_verbosity = True):
    fichier = os.path.join(répertoire_travail, nom_fichier) + '.gz'
    if os.path.isfile(fichier):
        if work_verbosity: print("Rien à faire. Le fichier est déjà présent.")
    else:
        requête_données = requests.get(url)
        if requête_données.status_code == 200:
            with open(fichier,mode='wb') as file:
             file.write(requête_données.content)
            if work_verbosity: print('Téléchargement : ', fichier)    
        else:
            print("Le lien ", url, "ne marche pas !")
            sys.exit()

# fonction pour décompresser les fichiers MF
def décompresser_archive(archive, work_verbosity = True):
    fichier             = os.path.join(répertoire_travail, archive) + '.gz'
    fichier_décompressé = os.path.join(répertoire_travail, archive) + '.csv'
    if os.path.isfile(fichier_décompressé):
        if work_verbosity: print("Rien à faire. Le fichier est déjà présent.")
    else:
        if os.path.exists(fichier):
            with gzip.open(fichier, mode='rb') as f_in:
                with open(fichier_décompressé, mode='wb') as f_out:
                    f_out.write(f_in.read())
            if work_verbosity: print('Archive décompressée : ', fichier)
        else:
            print("L'archive ", fichier, " n'est pas présente !")
            print("Téléchargez tout d'abord l'archive...")

######################################
# Extraction des informations brutes #
######################################
def extraction_information(dept : str):
      """ 
      pour chaque département, télécharge les fichiers de Météo France. On recherche :
            * les caractéristiques du poste, à savoir :
                  le nom, l'altitude, les coordonnées (latitude, longitude)
            * les dates de début et de fin des mesures
            * quel type de variables a été mesuré :
                  - pluie : pluie journalière
                  - neige : cumul de neige sur 24 h
                  - HN : épaisseur du manteau neigeux
                  - TM : température moyenne
                  - TN : température minimale
                  - TX : température maximale
            * le fichier d'origine de Météo-France (séquence) :
                  - 0 : météo (pluie, TM, TN, TX) avant 1950
                  - 1 : météo (pluie, TM, TN, TX) de 1950 à 2022
                  - 2 : météo (pluie, TM, TN, TX) après 2022
                  - 3 : nivo (neige, HN) avant 1950
                  - 4 : nivo (neige, HN) de 1950 à 2022
                  - 5 : nivo (neige, HN) après 2022
      La fonction fournit un fichier "fichier_postes"  (dataframe).
      Les données sont exportées si la variable globale export_détail = True.
      """
      # fichier_postes : liste des postes avec leur caractéristiques
      fichier_postes = list()
      # indice fichier : valeur de 0 à 5 avec 0 pour M1, 1 pour M2, 2 pour M3
      #                                       3 pour N1, 4 pour N2, 5 pour N3
      liste_url = [ url_M1_département[dept], url_M2_département[dept], url_M3_département[dept],
              url_N1_département[dept], url_N2_département[dept], url_N3_département[dept] ]
      # boucle pour chaque fichier M1-M3 puis N1-N3
      for indice_fichier in tqdm(range(6)):
            archive = 'test_'+str(indice_fichier)+'_'+dept
            télécharger_fichier( liste_url[indice_fichier], archive, work_verbosity = verbosity) 
            décompresser_archive(archive, work_verbosity = verbosity)
            fichier = os.path.join(répertoire_travail, archive) + '.csv'
            données_totales = pd.read_csv(fichier, sep=";", encoding= 'utf-8',parse_dates=["AAAAMMJJ"],dtype={"NUM_POSTE":str, 'AAAAMMJJ':str})
            # recense tous les postes et calcule le nombre d'enregistrements
            liste_poste = np.array( données_totales['NUM_POSTE'].value_counts().reset_index().rename(columns={"index": "poste", 0: "nb valeurs"}) )
            # ajoute le DataFrame à fichier postes
            fichier_postes.append(pd.DataFrame(columns=['poste', 'nom', 'valeurs', 'alt', 'lat', 'lon', 
                                                  'début','fin', 'pluie', 'TM', 'TN', 'TX', 'HN', 'neige', 'séquence'] ))
            # indice pour le poste
            for indice_poste in range(len(liste_poste)):
                  nombre_valeurs = liste_poste[indice_poste,1]
                  numéro_poste   = liste_poste[indice_poste,0]
                  première_ligne = données_totales[données_totales['NUM_POSTE'] == numéro_poste ].iloc[0]
                  nom_poste = première_ligne['NOM_USUEL'].title()
                  latitude  = première_ligne['LAT']
                  longitude = première_ligne['LON']
                  altitude  = première_ligne['ALTI']
                  date_début= première_ligne['AAAAMMJJ'].strftime("%Y-%m-%d")
                  dernière_ligne = données_totales[données_totales['NUM_POSTE'] == liste_poste[0,0] ].iloc[-1]
                  date_fin = dernière_ligne['AAAAMMJJ'].strftime("%Y-%m-%d")
                  # teste si toutes les données sont NaN ou non
                  if indice_fichier < 3:
                        existe_RR = not np.isnan(np.array(données_totales[données_totales['NUM_POSTE'] == liste_poste[indice_poste,0] ]['RR'])).all()
                        existe_TX = not np.isnan(np.array(données_totales[données_totales['NUM_POSTE'] == liste_poste[indice_poste,0] ]['TX'])).all()
                        existe_TM = not np.isnan(np.array(données_totales[données_totales['NUM_POSTE'] == liste_poste[indice_poste,0] ]['TM'])).all()
                        existe_TN = not np.isnan(np.array(données_totales[données_totales['NUM_POSTE'] == liste_poste[indice_poste,0] ]['TN'])).all()
                        existe_HN = False
                        existe_C1 = False
                  else:
                        existe_RR = False
                        existe_TX = False
                        existe_TM = False
                        existe_TN = False
                        existe_HN = not np.isnan(np.array(données_totales[données_totales['NUM_POSTE'] == liste_poste[indice_poste,0] ]['NEIGETOTX'])).all()
                        existe_C1 = not np.isnan(np.array(données_totales[données_totales['NUM_POSTE'] == liste_poste[indice_poste,0] ]['HNEIGEF'])).all()                  
                  dict1 = {'poste':[numéro_poste], 'nom':[nom_poste], 'valeurs':[nombre_valeurs], 
                          'alt':[altitude], 'lat':[latitude],'lon':[longitude],
                          'début':[date_début], 'fin':[date_fin], 'pluie':[existe_RR],
                          'TM':[existe_TM],'TN':[existe_TN],'TX':[existe_TX],
                          'HN':[existe_HN],'neige':[existe_C1],'séquence': [indice_fichier]} 
                  ajout     = pd.DataFrame(dict1)
                  fichier_postes[indice_fichier] = pd.concat([fichier_postes[indice_fichier],ajout])
            
      # assemblage
      fichier_postes=pd.concat(fichier_postes).sort_values('poste')
      # export du détail du département
      if export_détail:
            fichier_excel   = os.path.join(répertoire_travail, 'liste_détaillée_'+dept+'.xlsx')
            onglet_liste = 'liste'
            fichier_postes.to_excel(fichier_excel, index=False, sheet_name= onglet_liste)
      return fichier_postes

######################################
# Synthèse des informations brutes   #
######################################
def synthétiser_information(dept : str):
      """ 
      fournit une synthèse de l'information pour le département, à savoir :
            * le nom, l'altitude, les coordonnées (latitude, longitude)
            * les dates de début et de fin des mesures
            * quel type de variables a été mesuré :
                  - pluie : pluie journalière
                  - neige : cumul de neige sur 24 h
                  - HN : épaisseur du manteau neigeux
                  - TM : température moyenne
                  - TN : température minimale
                  - TX : température maximale
      La fonction fait appel à la fonction extraction_information
      La fonction fournit un fichier "liste_finale" (dataframe).
      Les données sont exportées si la variable globale export_liste = True.
      """
      fichier_postes = extraction_information(dept)
      liste_poste = np.array( fichier_postes['poste'].value_counts().reset_index())[:,0]
      liste_finale = pd.DataFrame(columns=['poste', 'nom', 'valeurs', 'alt', 'lat', 'lon', 
                                                  'début','fin', 'pluie', 'neige','HN','TM', 'TN', 'TX' ] )
      for poste in liste_poste:
            sélection_valeurs = fichier_postes[fichier_postes['poste']==poste]
            nombre_valeurs = sélection_valeurs['valeurs'].sum()
            numéro_poste   = poste
            première_ligne = sélection_valeurs.iloc[0]
            nom_poste = première_ligne['nom']
            latitude  = première_ligne['lat']
            longitude = première_ligne['lon']
            altitude  = première_ligne['alt']
            date_début= sélection_valeurs['début'].min()
            date_fin  = sélection_valeurs['fin'].max()
            existe_pluie = sélection_valeurs['pluie'].any()
            existe_neige = sélection_valeurs['neige'].any()
            existe_HN = sélection_valeurs['HN'].any()
            existe_TN = sélection_valeurs['TN'].any()
            existe_TM = sélection_valeurs['TM'].any()
            existe_TX = sélection_valeurs['TX'].any()
            dict1 = {'poste':[numéro_poste], 'nom':[nom_poste], 'valeurs':[nombre_valeurs], 
                          'alt':[altitude], 'lat':[latitude],'lon':[longitude],
                          'début':[date_début], 'fin':[date_fin], 'pluie':[existe_pluie],
                          'TM':[existe_TM],'TN':[existe_TN],'TX':[existe_TX],
                          'HN':[existe_HN],'neige':[existe_neige]} 
            ajout     = pd.DataFrame(dict1)
            liste_finale = pd.concat([liste_finale, ajout])
      liste_finale = liste_finale.sort_values(by=['nom'])
      # export de la synthèse du département
      if export_liste:
            fichier_excel   = os.path.join(répertoire_travail, 'liste_synthétique_'+dept+'.xlsx')
            onglet_liste = 'liste-stations'
            liste_finale.to_excel(fichier_excel, index=False, sheet_name= onglet_liste)
      return liste_finale

######################################
# Carte des stations                 #
######################################
def cartographier_postes(liste_p, dept: str):
      """ 
      Cartographie les postes.
      La carte est exportée si la variable globale export_carte = True.
      """
      m_lat=np.mean(np.array(liste_p.lat))
      m_lon=np.mean(np.array(liste_p.lon))
      aujourdhui = datetime.datetime.now()
      title = noms_département[dept]+" : postes actifs en " + aujourdhui.strftime("%Y")
      fig = px.scatter_mapbox(liste_p, 'lat', lon= 'lon', 
                        hover_name= "nom", hover_data= ['poste','alt', 'début', 'fin'], 
                        zoom=5, height= 800, width= 900, 
                        )
      fig.update_layout(title_text= title, title_x=0.5  , hoverlabel= dict(bgcolor= 'rgb(255,255,255)'))
      fig.update_traces(marker=dict(color='red'))
      #===================== Ajoute une couche des départements français à partir d'un fichier GeoJSON
      fig.update_layout(mapbox_layers=[
            {
            "sourcetype": "geojson",
            # "source": "https://france-geojson.gregoiredavid.fr/repo/departements.geojson",
            "source": "https://www.data.gouv.fr/fr/datasets/r/90b9341a-e1f7-4d75-a73c-bbc010c7feeb", 
            "type": "line",
            "color": "rgba(0, 0, 0, 0.9)",
            "line": {"width": 1}
            }
            ])
      fig.update_layout(mapbox_style= "open-street-map", mapbox_zoom=8, mapbox_center = {"lat": m_lat, "lon": m_lon})
      fig.show()
      if export_carte:
            # Export PNG
            file_png= os.path.join(répertoire_travail, 'carte_postes_'+dept + '.png')
            fig.write_image(file_png, height= 800, width= 900,   scale = 3)


J'ai trouvé le répertoire :  /home/ancey/Météo-France/postes/


# Extraction pour la Savoie

In [65]:
liste_73 = synthétiser_information('73')
cartographier_postes(liste_73, dept = '73')

100%|██████████| 6/6 [00:15<00:00,  2.58s/it]


# Extraction Alpes du Nord

In [71]:
départements_alpes_nord = ['73','74','38']

liste_alpes_nord = pd.DataFrame(columns=['poste', 'nom', 'valeurs', 'alt', 'lat', 'lon', 
                                                  'début','fin', 'pluie', 'neige','HN','TM', 'TN', 'TX' ] )
for dept in départements_alpes_nord:
      liste_départementale = synthétiser_information(dept)
      liste_alpes_nord = pd.concat([liste_alpes_nord, liste_départementale])
      

cartographier_postes(liste_alpes_nord, dept = 'nord')
      

100%|██████████| 6/6 [00:49<00:00,  8.17s/it]


In [72]:
liste_alpes_nord

,poste,nom,valeurs,alt,lat,lon,début,fin,pluie,neige,HN,TM,TN,TX
0,73001001,Aiguebelette,5539,400,45.551667,5.813333,1941-10-01,2022-12-31,True,False,False,False,False,False
0,73004001,Aillon Le Jeune,46217,900,45.617833,6.080167,1934-06-01,2022-12-31,True,True,True,False,False,False
0,73008003,Aix Les Bains,5296,234,45.700333,5.889667,2003-01-01,2022-12-31,True,False,False,False,False,False
0,73008001,Aix-Les-Bains,9048,365,45.693333,5.908333,1947-01-01,2022-12-31,True,False,False,False,True,True
0,73011002,Albertville,43171,331,45.662167,6.371833,1948-01-01,2022-12-31,True,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,38548002,Villard-De-Lans-Gend,19607,1031,45.070333,5.552333,1916-01-01,2022-12-31,True,False,False,False,True,True
0,38549001,Villard-Notre-Dame Village,5313,1550,45.018333,6.041667,1950-04-18,2022-12-31,True,False,False,False,False,False
0,38559004,Vinay,656,400,45.212000,5.376833,2015-03-01,2022-12-31,True,True,True,False,False,False
0,38559001,Vinay Bourg,1952,260,45.210000,5.401667,1949-01-01,2022-12-31,True,False,False,False,False,False


In [73]:
fichier_excel   = os.path.join(répertoire_travail, 'liste_synthétique_Alpes_Nord.xlsx')
onglet_liste = 'liste-stations'
liste_alpes_nord.to_excel(fichier_excel, index=False, sheet_name= onglet_liste)

# Extraction pour les Hautes Alpes

In [75]:
liste_05 = synthétiser_information('05')
cartographier_postes(liste_05, dept = '05')

100%|██████████| 6/6 [00:56<00:00,  9.38s/it]


In [76]:
liste_05

,poste,nom,valeurs,alt,lat,lon,début,fin,pluie,neige,HN,TM,TN,TX
0,05001001,Abries,35920,1560,44.796667,6.928333,1932-01-01,2024-05-30,True,False,False,False,True,True
0,05001001,Abries,1,1560,44.796667,6.928333,1948-12-15,1949-12-31,True,False,False,False,False,False
0,05001401,Abries -S-,1,2070,44.791333,6.949667,2007-01-01,2022-12-31,False,False,False,False,False,False
0,05001400,Abries Rm,5764,2018,44.792833,6.947000,1985-12-23,2024-05-30,True,True,True,False,True,True
0,05139006,Agnieres-En-Devoluy,51428,1244,44.696167,5.880833,1928-11-01,2024-05-30,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,05181002,Villar D'Arene,14451,1665,45.030833,6.361667,2004-07-31,2024-05-30,True,False,True,True,True,True
0,05182001,Villar Loubiere,47559,1072,44.826833,6.146000,1928-01-01,2024-05-30,True,True,True,True,True,True
0,05183001,Villar St Pancrace,15216,1310,44.880333,6.640333,2003-08-01,2024-05-30,True,True,True,True,True,True
0,05181001,Villar-D'Arene Village,5586,1650,45.041667,6.336667,1944-06-01,2022-12-31,True,False,False,False,False,False
